# 🎨 OpenAI GPT Image 1 API - 코랩 사용 가이드

이 노트북은 OpenAI의 최신 GPT Image 1 모델을 Google Colab에서 사용하는 방법을 보여줍니다.

## 주요 특징
- **gpt-image-1**: DALL-E 3보다 향상된 최신 모델
- **Input Fidelity**: 얼굴/로고 등 세부사항 고정밀 보존
- **High Quality**: 전문가급 이미지 생성

## 필요사항
- OpenAI API 키
- Organization 인증 (GPT Image 1 사용 시)

## 1️⃣ 환경 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install openai pillow requests

In [ ]:
import os
import requests
from openai import OpenAI
from IPython.display import Image, display
from PIL import Image as PILImage
import io

In [ ]:
# API 키 설정 (보안을 위해 Colab Secrets 사용 권장)
# Colab에서: 왼쪽 사이드바 > 🔑 아이콘 > API_KEY 추가

try:
    from google.colab import userdata
    api_key = userdata.get('OPENAI_API_KEY')
except:
    # 직접 입력 (보안 주의!)
    api_key = input("OpenAI API 키를 입력하세요: ")

os.environ['OPENAI_API_KEY'] = api_key
client = OpenAI(api_key=api_key)

print("✅ OpenAI 클라이언트 설정 완료")

## 2️⃣ 기본 이미지 생성 (GPT Image 1)

In [ ]:
# 기본 이미지 생성
def generate_image(prompt, size="1024x1024", quality="standard"):
    try:
        response = client.images.generate(
            model="gpt-image-1",
            prompt=prompt,
            size=size,
            quality=quality,
            n=1
        )
        
        image_url = response.data[0].url
        print(f"🎨 이미지 생성 완료: {prompt}")
        
        # 이미지 표시
        display(Image(url=image_url))
        return image_url
        
    except Exception as e:
        print(f"❌ 오류: {e}")
        return None

# 예시 실행
prompt = "귀여운 강아지가 공원에서 뛰어노는 모습, 디지털 아트 스타일"
image_url = generate_image(prompt)

## 3️⃣ High Input Fidelity 이미지 생성

In [ ]:
# High Fidelity 모드 - 얼굴/세부사항 정확히 보존
def generate_high_fidelity_image(prompt, input_image_url=None):
    try:
        # GPT Image 1의 고품질 생성
        response = client.images.generate(
            model="gpt-image-1",
            prompt=prompt,
            # input_fidelity="high",  # 입력 이미지가 있을 때 사용
            size="1024x1024",
            quality="hd",  # 고품질 모드
            n=1
        )
        
        image_url = response.data[0].url
        print(f"🎨 High Fidelity 이미지 생성: {prompt}")
        display(Image(url=image_url))
        return image_url
        
    except Exception as e:
        print(f"❌ 오류: {e}")
        return None

# 고품질 이미지 생성 예시
hd_prompt = "사실적인 초상화, 젊은 여성, 자연스러운 미소, 스튜디오 조명"
hd_image = generate_high_fidelity_image(hd_prompt)

## 4️⃣ 이미지 분석 (Vision API)

In [ ]:
# 이미지 분석 함수
def analyze_image(image_url, question="이 이미지에 대해 설명해주세요"):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": question},
                        {
                            "type": "image_url",
                            "image_url": {"url": image_url}
                        }
                    ]
                }
            ],
            max_tokens=500
        )
        
        analysis = response.choices[0].message.content
        print(f"🔍 이미지 분석 결과:")
        print(analysis)
        return analysis
        
    except Exception as e:
        print(f"❌ 분석 오류: {e}")
        return None

# 생성된 이미지 분석
if 'image_url' in locals() and image_url:
    analyze_image(image_url, "이 이미지의 구성 요소와 스타일을 분석해주세요")

## 5️⃣ 이미지 편집 (DALL-E 2)

In [ ]:
# 이미지 편집 (마스크 없이)
def edit_image_simple(original_url, edit_prompt):
    try:
        # 원본 이미지 다운로드
        response = requests.get(original_url)
        original_image = PILImage.open(io.BytesIO(response.content))
        
        # PNG로 변환 (투명도 추가)
        if original_image.mode != 'RGBA':
            original_image = original_image.convert('RGBA')
        
        # 임시 파일로 저장
        original_image.save('/tmp/original.png')
        
        # DALL-E 2로 편집
        with open('/tmp/original.png', 'rb') as image_file:
            edit_response = client.images.edit(
                model="dall-e-2",
                image=image_file,
                prompt=edit_prompt,
                n=1,
                size="1024x1024"
            )
        
        edited_url = edit_response.data[0].url
        print(f"✏️ 이미지 편집 완료: {edit_prompt}")
        display(Image(url=edited_url))
        return edited_url
        
    except Exception as e:
        print(f"❌ 편집 오류: {e}")
        return None

# 편집 예시 (이전에 생성한 이미지가 있다면)
if 'image_url' in locals() and image_url:
    edit_prompt = "배경을 아름다운 해변으로 바꿔주세요"
    edited_image = edit_image_simple(image_url, edit_prompt)

## 6️⃣ 배치 이미지 생성

In [ ]:
# 여러 이미지 생성
def generate_multiple_images(prompts, model="gpt-image-1"):
    results = []
    
    for i, prompt in enumerate(prompts):
        print(f"🎨 {i+1}/{len(prompts)} 생성 중: {prompt[:50]}...")
        
        try:
            response = client.images.generate(
                model=model,
                prompt=prompt,
                size="1024x1024",
                quality="standard",
                n=1
            )
            
            image_url = response.data[0].url
            results.append({"prompt": prompt, "url": image_url})
            
            # 이미지 표시
            print(f"✅ 완료: {prompt}")
            display(Image(url=image_url))
            
        except Exception as e:
            print(f"❌ 오류: {e}")
            results.append({"prompt": prompt, "error": str(e)})
    
    return results

# 동물 테마 이미지 생성
animal_prompts = [
    "귀여운 골든 리트리버 강아지, 햇살 가득한 정원에서",
    "우아한 페르시안 고양이, 창가에서 낮잠자는 모습",
    "활발한 토끼, 당근을 먹고 있는 모습, 만화 스타일"
]

animal_results = generate_multiple_images(animal_prompts)

## 7️⃣ 이미지 다운로드 및 저장

In [ ]:
# 이미지 다운로드 및 저장
def download_and_save_image(image_url, filename):
    try:
        response = requests.get(image_url)
        
        with open(filename, 'wb') as f:
            f.write(response.content)
        
        print(f"💾 이미지 저장 완료: {filename}")
        return filename
        
    except Exception as e:
        print(f"❌ 저장 오류: {e}")
        return None

# 생성된 이미지들 저장
if 'animal_results' in locals():
    for i, result in enumerate(animal_results):
        if 'url' in result:
            filename = f"generated_animal_{i+1}.png"
            download_and_save_image(result['url'], filename)

## 8️⃣ 고급 기능: Input Fidelity 테스트

In [ ]:
# High Input Fidelity 테스트 (조직 인증 필요)
def test_input_fidelity():
    prompts = [
        {"prompt": "프로필 사진, 전문적인 비즈니스 스타일", "fidelity": "standard"},
        {"prompt": "프로필 사진, 전문적인 비즈니스 스타일", "fidelity": "high"}
    ]
    
    for test in prompts:
        try:
            print(f"🔍 테스트: {test['fidelity']} fidelity")
            
            # gpt-image-1에서 input_fidelity 파라미터 사용
            response = client.images.generate(
                model="gpt-image-1",
                prompt=test['prompt'],
                # input_fidelity=test['fidelity'],  # 조직 인증 후 사용 가능
                size="1024x1024",
                quality="hd",
                n=1
            )
            
            image_url = response.data[0].url
            print(f"✅ {test['fidelity']} fidelity 생성 완료")
            display(Image(url=image_url))
            
        except Exception as e:
            print(f"❌ {test['fidelity']} fidelity 오류: {e}")
            if "organization" in str(e).lower():
                print("💡 Organization 인증이 필요합니다")

# Fidelity 테스트 실행
test_input_fidelity()

## 9️⃣ 프로젝트 연동 예시

In [ ]:
# ALL-I-PROJECT 동물 입양 플랫폼용 이미지 생성
def generate_adoption_images():
    adoption_prompts = [
        "보호소에서 기다리는 귀여운 강아지, 희망적인 표정, 따뜻한 조명",
        "입양을 기다리는 고양이, 장난스러운 모습, 밝은 배경",
        "행복한 가족과 새로 입양된 강아지, 집 앞마당에서",
        "동물 보호소 로비, 깨끗하고 따뜻한 분위기"
    ]
    
    adoption_images = []
    
    for prompt in adoption_prompts:
        image_url = generate_image(prompt)
        if image_url:
            adoption_images.append(image_url)
    
    return adoption_images

# 입양 플랫폼용 이미지 생성
adoption_images = generate_adoption_images()
print(f"\n🎉 총 {len(adoption_images)}개의 입양 관련 이미지가 생성되었습니다!")

## 🔧 유틸리티 함수들

In [ ]:
# 이미지 크기 조정
def resize_image(image_url, size=(512, 512)):
    try:
        response = requests.get(image_url)
        img = PILImage.open(io.BytesIO(response.content))
        
        resized = img.resize(size, PILImage.Resampling.LANCZOS)
        
        # 메모리에서 표시
        display(resized)
        return resized
        
    except Exception as e:
        print(f"❌ 크기 조정 오류: {e}")
        return None

# 비용 계산기 (근사치)
def calculate_cost(num_images, quality="standard", size="1024x1024"):
    # GPT Image 1 가격 (근사치)
    costs = {
        ("standard", "512x512"): 0.02,
        ("standard", "1024x1024"): 0.07,
        ("hd", "1024x1024"): 0.19
    }
    
    cost_per_image = costs.get((quality, size), 0.07)
    total_cost = num_images * cost_per_image
    
    print(f"💰 예상 비용: {num_images}개 이미지 × ${cost_per_image} = ${total_cost:.2f}")
    return total_cost

# 비용 계산 예시
calculate_cost(10, "standard", "1024x1024")

## 📝 사용 팁

### 1. API 키 보안
- Colab Secrets 사용 권장
- 절대 코드에 직접 입력 금지

### 2. 모델 선택
- **gpt-image-1**: 최신, 고품질 생성
- **dall-e-2**: 편집 기능 지원
- **dall-e-3**: 중간 성능

### 3. Input Fidelity
- `input_fidelity="high"`: 얼굴/세부사항 보존
- 추가 토큰 소모 (4096-6144)
- Organization 인증 필요

### 4. 비용 최적화
- Quality: standard < hd
- Size: 512x512 < 1024x1024
- 배치 처리로 효율성 증대